In [1]:
# run stopwatch
from _classes import Stopwatch
stopwatch = Stopwatch()
start = stopwatch.start()

### Load Saved Section if exists

### Load libraries, functions, palette, theme

In [2]:
%run _load.ipynb

In [3]:
economics_data = loadit(
    'economics_data', 'data/00-Datasets/'
)

In [4]:
dir = '008.1_SOL-Consumers-expectations'

In [5]:
save_df = 'data/00-Datasets/'

In [6]:
save_excel = 'data/00-Excel-data/'

# <font color='#2C8B6D'>I. Население</font>

# Section II. Уровень жизни

## 8. Потребительские ожидания населения

### 8.1. Индекс потребительской уверенности

*Источник: https://rosstat.gov.ru/folder/13397*  
*Единицы измерения: *

*Примечания:*  

In [7]:
consumer_confidence_link = \
    ('data/02-Population/02-Standard-of-living/05-Households/'
     'urov_81.xls')

In [8]:
consumer_confidence_raw = pd.read_excel(consumer_confidence_link)

In [9]:
consumer_confidence_raw.head(10)

,Обновлено 20.09.2023 г.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Индекс потребительской уверенности,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,баланс %,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Всего,в том числе по полу,NaN,в том числе по возрасту,NaN,NaN,NaN,NaN
6,NaN,NaN,мужчины,женщины,до 30 лет,30-49 лет,50 лет и старше,NaN,NaN
7,1998 год,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,IV квартал,-58,-57,-59,-51,-59,-62,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
consumer_confidence = consumer_confidence_raw.iloc[7:, :2].copy()

In [11]:
# rename columns
consumer_confidence.columns = ['Год', 'Значение']

In [12]:
# remove spaces from start and end of values in 'Год'
consumer_confidence['Год'] = [i.strip() if isinstance(i, str) else i for i in consumer_confidence['Год']]

In [13]:
# remove NaNs from column 'Год'
consumer_confidence = consumer_confidence[~consumer_confidence['Год'].isna()]

In [14]:
# remove ' год' from year values
consumer_confidence['Год'] = consumer_confidence['Год'].replace(' год', '', regex=True)

In [15]:
# transform to int year values in 'Год'
consumer_confidence['Год'] = [to_int(i) for i in consumer_confidence['Год']]

In [16]:
# transform to float 'Значение' values if possible
consumer_confidence['Значение'] = [to_float(i) for i in consumer_confidence['Значение']]

In [17]:
# create list with years - take only ints from 'Год' column
years_list = [i for i in consumer_confidence['Год'] if isinstance(i, int)]

In [18]:
# create 'index' column
consumer_confidence['index'] = consumer_confidence['Год'].copy()

In [19]:
# calculate sum of 'квартал' rows and assign it to cell with .loc[year, 'Значение']

# for every index in 'consumer_confidence'
for index in consumer_confidence.index:
    # if meet index == year
    if consumer_confidence.loc[index, 'Год'] in years_list:
        # create list with indexes to count mean
        indexes_for_mean = []
        # moving down by indexes after current year index
        for index2 in consumer_confidence.loc[index+1:, :].index:
            # until meet next year
            if consumer_confidence.loc[index2, 'Год'] not in years_list:
                # add index to list for count mean
                indexes_for_mean.append(index2)
                # now, till we meet next year index, we have list with indexes,
                # that belong for 1,2,3,4 quartal of meeted year
            else:
                # then we calculate average value
                average = consumer_confidence.loc[indexes_for_mean, 'Значение'].mean()
                # add it to 'Значение' of current year
                consumer_confidence.loc[index, 'Значение'] = average
                # and break second cycle - moving through indexes, searching another year from years_list
                break
        # if we meet last year of years_list
        if consumer_confidence.loc[index, 'Год'] == years_list[-1]:
            # then just calculate average
            average = consumer_confidence.loc[index:, 'Значение'].mean()
            average = np.round(average, 2)
            # and add it to current year 'Значение'
            consumer_confidence.loc[index, 'Значение'] = average
    # if value not in year_list, then change it in 'index' column to NaN
    else:
        consumer_confidence.loc[index, 'index'] = np.NaN

In [20]:
consumer_confidence = transform_fill_values_by_previous(
    data=consumer_confidence, kind='column', column_name='index')

In [21]:
consumer_confidence.loc[consumer_confidence['Год'].isin(years_list), 'Год'] = 'Всего'

In [22]:
consumer_confidence = consumer_confidence.set_index('index', drop=True)
consumer_confidence.index.name = None

In [23]:
consumer_confidence.head(10)

,Год,Значение
1998,Всего,-58.00
1998,IV квартал,-58.00
1999,Всего,-45.25
1999,I квартал,-51.00
1999,II квартал,-48.00
1999,III квартал,-47.00
1999,IV квартал,-35.00
2000,Всего,-19.25
2000,I квартал,-25.00
2000,II квартал,-17.00


##### <font color='#AF4035'> Export Dataset to Excel

In [24]:
saveit_excel(
    data=consumer_confidence,
    filename=dir,
    path=save_excel,
    sheet='Потребительская уверенность'
)

File '010206-SOL-Consumers-expectations' created


### Save Dataset

In [25]:
economics_data['8.1'] = consumer_confidence

In [26]:
saveit(
    file=economics_data,
    name='economics_data',
    dir=save_df
)

File 'economics_data' saved in directory 'data/00-Datasets/'


### Save Session

In [27]:
save_session(dir)

### Execution time

In [28]:
print(f'Execution time: {stopwatch.stop(start)}')

Execution time: 0:00:10
